In [1]:
from vecstack import stacking
from sklearn.ensemble import StackingClassifier
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

In [2]:
RANDOM_SEED = 42

os.environ['PYTHONHASHSEED']=str(RANDOM_SEED)

random.seed(RANDOM_SEED)

np.random.seed(RANDOM_SEED)

In [3]:
#métricas
modelos = []
acuracia = []
precisao = []
sensibilidade = []
especificidade = []
f1_score = []

In [4]:
def get_answers():
  for i in range(0, len(acuracia)):
    print("modelo: " + str(modelos[i]) + " possui acuracia de " + str(acuracia[i]))

In [5]:
#métricas de classificação
def metricas( list_pred, modelo):
  for i in range(0,len(list_pred)):
    cm = confusion_matrix(y_test, list_pred[i])

    acc = (cm[0,0]+cm[1,1])/(cm[0,0]+cm[0,1]+cm[1,0]+cm[1,1])
    prec = cm[1,1]/(cm[0,1]+cm[1,1])
    sens = cm[1,1]/(cm[1,0]+cm[1,1])
    esp=cm[0,0]/(cm[0,0]+cm[0,1])
    f1 = 2*prec*sens/(precisao+sensibilidade)

    modelos.append(modelo[i]), acuracia.append(acc), precisao.append(prec), sensibilidade.append(sens), especificidade.append(esp), f1_score.append(f1)

In [6]:
mitbih_train = pd.read_csv('mitbih_train.csv', header=None)
mitbih_test = pd.read_csv('mitbih_test.csv', header=None)



# Variáveis para treino
X_train = mitbih_train.iloc[:, :-1].values
y_train = mitbih_train.iloc[:, -1].values

# Variáveis para teste e validação (50% teste - 50% validação)
X_test, X_val, y_test, y_val = train_test_split(mitbih_test.iloc[:, :-1].values,
                                                mitbih_test.iloc[:, -1].values,
                                                test_size=0.5,
                                                random_state=RANDOM_SEED)

#X_test = mitbih_test.iloc[:, :-1].values
#y_test = mitbih_test.iloc[:, -1].values

In [7]:
# initializing all the base model objects with default parameters
model_1 = RandomForestClassifier(random_state=42)
model_2 = GradientBoostingClassifier(random_state=42)
model_3 = SVC(random_state=42)
model_4 = MLPClassifier(random_state=42)
model_5 = KNeighborsClassifier(n_neighbors=5)
model_6 = AdaBoostClassifier(random_state=42)


# putting all base model objects in one list
all_models = [model_1, model_2, model_3, model_4, model_5, model_6]

# computing the stack features
s_train, s_test = stacking(all_models, X_train, y_train,  X_test, regression=False, n_folds=5, shuffle=True, verbose=2)

# initializing the second-level model
#model_stacking = LogisticRegression()
model_stacking = xgb.XGBClassifier(random_state=42)

# fitting the second level model with stack features
model_stacking = model_stacking.fit(s_train, y_train)

# predicting the final output using stacking
ypred_stacking = model_stacking.predict(s_test)


list_pred = [ypred_stacking]
modelo = ['stacking']
metricas(list_pred, modelo)
get_answers()

task:         [classification]
n_classes:    [5]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [6]

model  0:     [RandomForestClassifier]
    fold  0:  [0.97236023]
    fold  1:  [0.97618640]
    fold  2:  [0.97493004]
    fold  3:  [0.97595797]
    fold  4:  [0.97595660]
    ----
    MEAN:     [0.97507825] + [0.00142709]
    FULL:     [0.97507824]

model  1:     [GradientBoostingClassifier]
    fold  0:  [0.96225230]
    fold  1:  [0.96447947]
    fold  2:  [0.96470790]
    fold  3:  [0.96293758]
    fold  4:  [0.96527698]
    ----
    MEAN:     [0.96393085] + [0.00114201]
    FULL:     [0.96393083]

model  2:     [SVC]
    fold  0:  [0.96430815]
    fold  1:  [0.96881960]
    fold  2:  [0.96733482]
    fold  3:  [0.96687796]
    fold  4:  [0.96938892]
    ----
    MEAN:     [0.96734589] + [0.00177723]
    FULL:     [0.96734587]

model  3:     [MLPClassifier]


/opt/homebrew/anaconda3/envs/TCC/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


    fold  0:  [0.97344526]


/opt/homebrew/anaconda3/envs/TCC/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


    fold  1:  [0.97590086]


...

    fold  2:  [0.84723888]


/opt/homebrew/anaconda3/envs/TCC/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


    fold  3:  [0.87145223]


/opt/homebrew/anaconda3/envs/TCC/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


    fold  4:  [0.87567105]
    ----
    MEAN:     [0.86233654] + [0.01016527]
    FULL:     [0.86233639]

modelo: stacking possui acuracia de 0.9880529544720698


In [8]:
for model in all_models:
    model.fit(X_train, y_train)

s_val = np.column_stack([model.predict(X_val) for model in all_models])

y_val_pred = model_stacking.predict(s_val)
classification_report_val = classification_report(y_val, y_val_pred, output_dict=True, zero_division=0)

y_test_pred = model_stacking.predict(s_test)
classification_report_test = classification_report(y_test, y_test_pred, output_dict=True, zero_division=0)

print("Relatório de Classificação - Validação (Stacking)")
print(classification_report(y_val, y_val_pred, zero_division=0))

print("Relatório de Classificação - Teste (Stacking)")
print(classification_report(y_test, y_test_pred, zero_division=0))

cm_test = confusion_matrix(y_test, y_test_pred)
cm_val = confusion_matrix(y_val, y_val_pred)

specificities_test = []
for i in range(len(cm_test)):
    TN = np.sum(cm_test) - np.sum(cm_test[i, :]) - np.sum(cm_test[:, i]) + cm_test[i, i]
    FP = np.sum(cm_test[:, i]) - cm_test[i, i]
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    specificities_test.append(specificity)

specificity_mean_test = np.mean(specificities_test)
print(f"Especificidade Média do Stacking (Teste): {specificity_mean_test:.4f}")

specificities_val = []
for i in range(len(cm_val)):
    TN = np.sum(cm_val) - np.sum(cm_val[i, :]) - np.sum(cm_val[:, i]) + cm_val[i, i]
    FP = np.sum(cm_val[:, i]) - cm_val[i, i]
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    specificities_val.append(specificity)

specificity_mean_val = np.mean(specificities_val)
print(f"Especificidade Média do Stacking (Validação): {specificity_mean_val:.4f}")


/opt/homebrew/anaconda3/envs/TCC/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/anaconda3/envs/TCC/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Relatório de Classificação - Validação (Stacking)
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      9106
         1.0       0.96      0.66      0.78       267
         2.0       0.97      0.92      0.94       694
         3.0       0.83      0.60      0.69        84
         4.0       0.99      0.97      0.98       795

    accuracy                           0.98     10946
   macro avg       0.95      0.83      0.88     10946
weighted avg       0.98      0.98      0.98     10946

Relatório de Classificação - Teste (Stacking)
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      9012
         1.0       0.96      0.64      0.77       289
         2.0       0.98      0.91      0.94       754
         3.0       0.88      0.72      0.79        78
         4.0       0.99      0.97      0.98       813

    accuracy                           0.98     10946
   macro avg       0.96      0.85  